# Learning Curve for Multi-Spike Tempotron

## 1. Run the function-containing cells below

In [1]:
import numpy as np
from main_gen import gen_features

def gen_background_data(n, fr, duration): 
    dt = 0.001 #bin (s)
    gen_bg = np.random.random((n, np.rint(duration/dt).astype(int)))<fr*dt
    gen_bg = gen_bg.astype(int)
    return gen_bg

def kernel_fn(length, tau_mem, tau_syn, time_ij):
    time = np.arange(0., length, 1.) #ms
    kernel = np.zeros(length)
    eta = tau_mem/tau_syn
    V_norm = eta**(eta/(eta-1))/(eta-1)
    for count in range(length):
        kernel[count] = V_norm*(np.exp(-(time[count]-time_ij)/tau_mem)-np.exp(-(time[count]-time_ij)/tau_syn))
    return kernel

In [4]:
### From voltage calculation code by Alex ###

def get_memory_len(kernel_array, ratio):
    arr = (kernel_array - ratio*kernel_array.max())[::-1]
    memory_len = len(kernel_array) - np.searchsorted(arr, 0)
    return memory_len

def presyn_input(data):
    datalen = data.shape[1]
    presyn_input = np.zeros(data.shape)
    for neuron, ith_bin in zip(*np.where(data)):
        mem_len = min(syn_memory_len, datalen - ith_bin)
        presyn_input[neuron,ith_bin:ith_bin+mem_len] += syn_kernel[:mem_len]
    return presyn_input

## 2. Preset Parameters

Set the required parameters, then run the cell below to generate features (in dictionary form), postsynaptic potential (syn_kernel), and spike-triggered reset (ref_kernel). 

In [2]:
n = 500
n_fea = 10 #number of different features
fr = 5 #average firing rate of each input neuron (Hz)
T_fea = 0.05 #T_fea: feature duration (s)
dt = 0.001 #bin: discrete time in second
tau_mem = 20
tau_syn = 5
time_ij = 0
init_kernel_len = 200

np.random.seed(1000000000)
features = gen_features(n_fea, n, fr, dt, T_fea)

syn_kernel = kernel_fn(init_kernel_len, tau_mem, tau_syn, time_ij)
syn_memory_len = get_memory_len(syn_kernel, ratio=0.001)
syn_kernel = syn_kernel[:syn_memory_len]

ref_kernel = np.exp(- np.arange(1000) / tau_mem)
ref_memory_len = get_memory_len(ref_kernel, ratio=0.001)
ref_kernel = ref_kernel[:ref_memory_len]

## 3. Generate Unweighted Inputs

Run the cell below to generate unweighted inputs for n probe trials.

DO NOT run this cell if you already have the file containing generated probe trials (learning_curve_inputs.npy)!

In [ ]:
n_probe_trial = 100
learning_curve_inputs = []

for i in range(n_probe_trial): 
    np.random.seed(i*100000)
    bg_data = gen_background_data(n, fr, 1.95)
    data_null = np.insert(bg_data, 975, np.zeros((50, n)), axis = 1)
    syn_input_null = presyn_input(data_null)
    learning_curve_inputs.append(syn_input_null)

np.save("learning_curve_inputs", learning_curve_inputs)

Run the cell below to generate unweighted inputs for features.

DO NOT run this cell if you already have the 'feature_inputs.npy' file!

In [4]:
feature_inputs = []

for feature, spikes in features.items():
    new_spikes = np.append(spikes, np.zeros((n, 150)), axis = 1)
    fea_input = presyn_input(new_spikes)
    feature_inputs.append(fea_input)

np.save("feature_inputs", feature_inputs)

## 4. Computing Neural Responses

Run the function-containing cells below. Uncomment the 'np.savez()' lines in neural_response function if the input data is large to avoid losing of computed results when encountering problems.

In [7]:
def add_fea(n_inputs, fea_input, T_fea):
    start = (n_inputs.shape[2] - T_fea*1000)/2
    for n in n_inputs:
        n[:, start:start+fea_input.shape[1]] += fea_input
    return n_inputs

In [8]:
def neural_response(omega_list, n_unweighted_inputs, fea_inputs, T_fea, theta):
    R_fea_list = []
    tot_input = (omega_list[:, np.newaxis, :, np.newaxis] * n_unweighted_inputs[np.newaxis, :, :, :]).sum(axis = 2)
    null_spike_count = np.zeros(tot_input.shape[:-1])
    for ith_bin in range(tot_input.shape[-1]):
        null_spike_mask = tot_input[:,:,ith_bin] >= theta
        null_spike_count += null_spike_mask
        mem_len = min(ref_memory_len, tot_input.shape[-1] - ith_bin)
        tot_input[:,:,ith_bin:ith_bin+mem_len] += null_spike_mask[:,:,np.newaxis] * ref_kernel[:mem_len]
    R_null_list = np.mean(null_spike_count, axis = 1)/(n_unweighted_inputs.shape[2]/1000)
    #np.savez("R_null_list", R_null_list, null_spike_count)
    for feature in fea_inputs:
        fea_unweighted_inputs = add_fea(n_unweighted_inputs, feature, T_fea)
        fea_tot_input = (omega_list[:, np.newaxis, :, np.newaxis] * fea_unweighted_inputs[np.newaxis, :, :, :]).sum(axis = 2) 
        spike_count = np.zeros(fea_tot_input.shape[:-1])
        for ith_bin in range(fea_tot_input.shape[-1]):
            spike_mask = fea_tot_input[:,:,ith_bin] >= theta
            spike_count += spike_mask
            mem_len = min(ref_memory_len, fea_tot_input.shape[-1] - ith_bin)
            fea_tot_input[:,:,ith_bin:ith_bin+mem_len] += spike_mask[:,:,np.newaxis] * ref_kernel[:mem_len]
        fea_spike_count = spike_count - null_spike_count
        R_fea_list.append(np.mean(fea_spike_count, axis = 1))
        #np.savez("R_fea_list", R_fea_list)
    return R_fea_list, R_null_list

Run the cell below to compute the neural responses, which will be used to plot learning curves.

Filenames for the computed synaptic efficacies (omega) of different trainings are as follows:

a. 1 feature 1 spike: latest_omega1F1S_list.npy

b. 1 feature 5 spikes: latest_omega1F5S_list.npy

c. 5 features 1 spike: latest_omega5F1S_list.npy

d. 5 features different spikes: latest_omega5FmS_list.npy

e. 5 features 5 spikes: latest_omega5F5S_list.npy

In [3]:
omega = np.load("latest_omega1F1S_list.npy") # updated omega list after training
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 100)
fea_inputs = np.load("feature_inputs.npy") # unweighted inputs of features
theta = 1 # threshold

R_fea_list, R_null_list = neural_response(omega, l_curve_inputs, fea_inputs, T_fea, theta)
np.savez("neural_responses", R_fea_list, R_null_list)